In [ ]:
import numpy as np
import os 
import glob
from PIL import Image
import pandas as pd
import pandas as pd2
import pandas as pd3
import math

##Reading images
FaceImageList = []
NonfaceImageList = []

for f in glob.glob('faces\*.png'):
    im=Image.open(f)
    FaceImageList.append(im)
FaceImageListSize=len(FaceImageList)


for nf in glob.glob('non-faces\*.png'):
    im=Image.open(nf)
    NonfaceImageList.append(im)
NonfaceImageListSize=len(NonfaceImageList)
     
#print(FaceImageListSize,NonfaceImageListSize)

##Generating integral image
def generate_integralimage(image,height,width):
    for r in range(height):
        for c in range(width):
            ii=0
            for i in range(r+1):
                for j in range(c+1):
                    ii=ii+image[i][j]
            IntegralImage[r][c]=ii            
    return IntegralImage

##Build feature blocks
def build_features(integralimage,x,y,width,height,featuretype):
    result = None
    if featuretype == 1:
        A = ( x - 1,y - 1)
        B = (x - 1 , y + width - 1)
        C = (x + height//2 - 1, y - 1)
        D = ( x +  height//2 - 1,  y +  width - 1)
        E = ( x +  height - 1,  y - 1)
        F = ( x +  height - 1,  y +  width - 1)
        result = 2 * integralimage[D[0]][D[1]] + integralimage[A[0]][A[1]] - integralimage[B[0]][B[1]] - 2 * integralimage[C[0]][C[1]] + integralimage[E[0]][E[1]] - integralimage[F[0]][F[1]]

    elif featuretype == 2:
        A = ( x - 1,  y - 1)
        B = ( x - 1 ,  y +  width//2 - 1)
        C = ( x - 1 ,  y +  width - 1)
        D = ( x  +  height - 1,  y - 1)
        E = ( x  +  height - 1 ,  y +  width//2 - 1)
        F = ( x  +  height - 1 ,  y +  width - 1)
        result = 2 * integralimage[B[0]][B[1]] + integralimage[F[0]][F[1]] - integralimage[C[0]][C[1]] - 2 * integralimage[E[0]][E[1]] + integralimage[D[0]][D[1]] - integralimage[D[0]][D[1]]

    elif  featuretype == 3:
        A = ( x - 1,  y - 1)
        B = ( x - 1 ,  y +  width//3 - 1)
        C = ( x - 1 ,  y +  width//3 * 2 - 1)
        D = ( x - 1 ,  y +  width - 1)
        E = ( x  +  height - 1 ,  y - 1)
        F = ( x  +  height - 1 ,  y +  width//3 - 1)
        G = ( x  +  height - 1,  y +  width//3 * 2 - 1)
        H = ( x  +  height - 1,  y +  width - 1)
        result = 2 * integralimage[B[0]][B[1]] + 2 * integralimage[G[0]][G[1]] - 2 * integralimage[C[0]][C[1]] - 2 * integralimage[F[0]][F[1]] - integralimage[H[0]][H[1]] - integralimage[A[0]][A[1]] + integralimage[D[0]][D[1]] + integralimage[E[0]][E[1]]

    elif  featuretype == 4:
        A = ( x - 1,  y - 1)
        B = ( x - 1 ,  y +  width - 1)
        C = ( x +  height//3 - 1 ,  y - 1)
        D = ( x +  height//3 - 1 ,   y +  width - 1)
        E = ( x  +   height//3 * 2 - 1 ,  y - 1)
        F = ( x  +  height//3 * 2 - 1 ,  y +  width - 1)
        G = ( x  +  height - 1,  y - 1)
        H = ( x  +  height - 1,  y +  width - 1)
        result = 2 * integralimage[C[0]][C[1]] + 2 * integralimage[F[0]][F[1]] - 2 * integralimage[D[0]][D[1]] - 2 * integralimage[E[0]][E[1]] - integralimage[H[0]][H[1]] - integralimage[A[0]][A[1]] + integralimage[B[0]][B[1]] + integralimage[G[0]][G[1]]

    elif  featuretype == 5:
        A = ( x - 1,  y - 1)
        B = ( x - 1 ,  y +  width//2 - 1)
        C = ( x - 1 ,  y +  width - 1)
        D = ( x +  height//2 - 1,  y - 1)
        E = ( x +  height//2 - 1,  y +  width//2 - 1)
        F = ( x +  height//2 - 1,  y +  width - 1)
        G = ( x +  height - 1,  y - 1)
        H = ( x +  height - 1,  y +  width//2 - 1)
        I = ( x +  height - 1,  y +  width - 1)
        result = -integralimage[A[0]][A[1]] + 2 * integralimage[B[0]][B[1]] - integralimage[C[0]][C[1]] + 2 * integralimage[D[0]][D[1]] - 4 * integralimage[E[0]][E[1]] + 2 * integralimage[F[0]][F[1]] - integralimage[G[0]][G[1]] + 2 * integralimage[H[0]][H[1]] - integralimage[I[0]][I[1]]

    return(result)


##Getting integral images for face and non face lists
global imgwidth,imgheight
global IntegralImageList
maxfaces = 10000

IntegralImageList=[0]*(FaceImageListSize+NonfaceImageListSize)
ImageVector = [[[0]*6]*maxfaces for i in range(FaceImageListSize+NonfaceImageListSize)]

for im in range(FaceImageListSize+NonfaceImageListSize):
        
    if(im<FaceImageListSize):
        img=FaceImageList[im]
    else:
        img=NonfaceImageList[im-FaceImageListSize]
        
    imagefromlist = np.asarray(img)
    imgwidth, imgheight = img.size        

    IntegralImage=np.asarray([[0]*imgwidth]*imgheight)
    IntegralImageList[im]=generate_integralimage(imagefromlist,imgwidth,imgheight)
    
#print(len(IntegralImageList))

##extract features
F1Count=0
F2Count=0
F3Count=0
F4Count=0
F5Count=0

##for feature 1 - two vertical blocks
for i in range(imgheight+1):
    for j in range(imgwidth+1):
        for k in range(3,6+1):
            for l in range(3,6+1):
                
                if(((j+((2*k)-1))<imgwidth)and((i+l-1)<imgheight)):
                    im=0
                    for im in range (FaceImageListSize+NonfaceImageListSize):
                        f1 = build_features(IntegralImageList[im],i,j,2*k,l,1)
                        ImageVector[im][F1Count]=[1,i,j,k,l,f1]
                    F1Count=F1Count+1
print("There are ",F1Count," Type 1 features")
##for feature 2 - two horizontal blocks
F2Count=F1Count                    
for i in range(imgheight):
    for j in range(imgwidth):
        for k in range(2,4+1):
            for l in range(2,4+1):
                
                if(((j+k-1)<imgwidth)and((i+(2*l)-1)<imgheight)):
                    im=0
                    for im in range (FaceImageListSize+NonfaceImageListSize):       
                        f2 = build_features(IntegralImageList[im],i,j,k,2*l,2)
                        ImageVector[im][F2Count]=[2,i,j,k,l,f2]
                    F2Count=F2Count+1
print("There are ",F2Count-F1Count," Type 2 features")                    
##for feature 3 - three vertical blocks
F3Count=F2Count                 
for i in range(imgheight):
    for j in range(imgwidth):
        for k in range(2,6+1):
            for l in range(4,8+1):
                
                if(((j+(3*k)-1)<imgwidth)and((i+l-1)<imgheight)):
                    im=0
                    for im in range (FaceImageListSize+NonfaceImageListSize):  
                        f3 = build_features(IntegralImageList[im],i,j,3*k,l,3)
                        ImageVector[im][F3Count]=[3,i,j,k,l,f3]
                    F3Count=F3Count+1
print("There are ",F3Count-F2Count," Type 3 features")                    
##for feature 4 - three horizontal blocks
F4Count=F3Count                    
for i in range(imgheight):
    for j in range(imgwidth):
        for k in range(4,8+1):
            for l in range(2,4+1):
                
                if(((j+k-1)<imgwidth)and((i+(3*l)-1)<imgheight)):
                    im=0
                    for im in range (FaceImageListSize+NonfaceImageListSize):   
                        f4 = build_features(IntegralImageList[im],i,j,k,3*l,4)
                        ImageVector[im][F4Count]=[4,i,j,k,l,f4]
                    F4Count=F4Count+1
print("There are ",F4Count-F3Count," Type 4 features")                    
##for feature 5 - four blocks
F5Count=F4Count                    
for i in range(imgheight):
    for j in range(imgwidth):
        for k in range(2,4+1):
            for l in range(4,8+1):
                
                if(((j+(2*k)-1)<imgwidth)and((i+(2*l)-1)<imgheight)):
                    im=0
                    for im in range (FaceImageListSize+NonfaceImageListSize):     
                        f5 = build_features(IntegralImageList[im],i,j,2*k,2*l,5)
                        ImageVector[im][F5Count]=[5,i,j,k,l,f5]
                    F5Count=F5Count+1
                    
print("There are ",F5Count-F4Count," Type 5 features")

##Writing feature data to csv
Featurevectors=[[0]*(FaceImageListSize+NonfaceImageListSize) for i in range(F5Count)]

for i in range(F5Count):
    for j in range(FaceImageListSize+NonfaceImageListSize):
        Featurevectors[i][j]=ImageVector[j][i][5]
print("The total number of Haar features is ",len(Featurevectors))     

Label=[None]*(FaceImageListSize+NonfaceImageListSize)

for i in range(FaceImageListSize+NonfaceImageListSize):
    if(i<FaceImageListSize):
        Label[i]=1
    else:
        Label[i]=-1 

Weight=[None]*(FaceImageListSize+NonfaceImageListSize)

for i in range(FaceImageListSize+NonfaceImageListSize):
    if(i<FaceImageListSize):
        Weight[i]=1/(2*FaceImageListSize)
    else:
        Weight[i]=1/(2*NonfaceImageListSize)

FeatureData=pd.DataFrame()

for i in range(F5Count):
    FeatureData.insert(loc=i, column=i,value=Featurevectors[i])

FeatureData.insert(loc=F5Count,column=F5Count,value=Label)
FeatureData.insert(loc=F5Count+1,column=F5Count+1,value=Weight)

FeatureData.to_csv('FeatureData.csv')

FeatureRef=pd2.DataFrame()
FeatureRef.insert(loc=0,column=0,value=ImageVector[0])

FeatureRef.to_csv('FeatureLocation.csv')


